## Fake datas Generation

In [3]:
# generate_data.py
import pandas as pd
import numpy as np

# Create 100 fake users with traits (0-1 scale)
np.random.seed(42)
data = {
    "user_id": range(100),
    "extroverted": np.random.rand(100).round(2),
    "romantic": np.random.rand(100).round(2),
    "adventurous": np.random.rand(100).round(2),
    "prefers_extroverted": np.random.rand(100).round(2),
    "prefers_romantic": np.random.rand(100).round(2),
}
df = pd.DataFrame(data)
df.to_json("user_data.json", orient="records")
print("Fake data saved to user_data.json!")
df

Fake data saved to user_data.json!


,user_id,extroverted,romantic,adventurous,prefers_extroverted,prefers_romantic
0,0,0.37,0.03,0.64,0.05,0.10
1,1,0.95,0.64,0.08,0.53,0.90
2,2,0.73,0.31,0.16,0.54,0.51
3,3,0.60,0.51,0.90,0.64,0.83
4,4,0.16,0.91,0.61,0.73,0.32
...,...,...,...,...,...,...
95,95,0.49,0.35,0.52,0.93,0.35
96,96,0.52,0.73,0.77,0.86,0.58
97,97,0.43,0.90,0.22,0.43,0.08
98,98,0.03,0.89,0.62,0.75,0.97


## Application of the method FCMPY

In [4]:
# matching_engine.py
from fcmpy import FcmSimulator
import json

# Define the dating compatibility FCM
fcm_rules = {
    "extroverted": {"match_score": 0.6},
    "romantic": {"match_score": 0.9},
    "adventurous": {"match_score": 0.3},
    "prefers_extroverted": {"match_score": -0.5},  # Negative if they dislike extroversion
    "prefers_romantic": {"match_score": 0.7},
    "match_score": {}  # Output node
}

def calculate_compatibility(user1, user2):
    # Blend user traits and preferences
    initial_state = {
        "extroverted": (user1["extroverted"] + user2["extroverted"]) / 2,
        "romantic": (user1["romantic"] + user2["romantic"]) / 2,
        "prefers_extroverted": user1["prefers_extroverted"] * user2["extroverted"],
        "prefers_romantic": user1["prefers_romantic"] * user2["romantic"],
    }
    simulator = FcmSimulator()
    results = simulator.simulate(fcm_rules, initial_state, iterations=10)
    return results["match_score"][-1]  # Final compatibility score

# Load fake data
with open("user_data.json") as f:
    users = json.load(f)

# Match user 0 with everyone else (example)
matches = []
for target_user in users[1:]:
    score = calculate_compatibility(users[0], target_user)
    matches.append({"user_id": target_user["user_id"], "score": score})

# Save results
with open("match_results.json", "w") as f:
    json.dump({"user": users[0]["user_id"], "matches": matches}, f)

ModuleNotFoundError: No module named 'fcmpy'

## Vizualise Top Matching

In [3]:
# plot_results.py
import matplotlib.pyplot as plt
import json

# Load results
with open("match_results.json") as f:
    data = json.load(f)

# Sort matches by score
matches = sorted(data["matches"], key=lambda x: x["score"], reverse=True)[:5]

# Plot top 5 matches
plt.figure(figsize=(10, 5))
plt.bar([f"User {m['user_id']}" for m in matches], [m["score"] for m in matches])
plt.title(f"Top Matches for User {data['user']}")
plt.ylabel("Compatibility Score")
plt.ylim(0, 1)
plt.savefig("top_matches.png")
plt.show()